In [11]:
import numpy as np
import pandas as pd
import re
import math
import itertools
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import collections 

from ast import literal_eval
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import namedtuple
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
from  scipy import spatial
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

In [12]:
d2v_model = Doc2Vec.load('../model/word_embedding/Doc2vec_new_small2_4.model')

# modeling

### 데이터셋 불러오기

In [111]:
final_list  = pd.read_excel('./new_data/final_list_score7.xlsx')
inv_list  = pd.read_excel('./new_data/invest_df.xlsx')
final_list.fillna('nan', inplace=True)
final_list['token'] = final_list.token.apply(lambda x:literal_eval(x))

In [112]:
final_list['invest'] = [str(x).split(',') for x in final_list['invest']]

In [113]:
invest = list(set([el.strip() for el in final_list['invest'].sum()]))
inv_list.set_index('invest', inplace=True)
inv_list = inv_list.apply(minmax_scale)

In [114]:
final_list.shape

(102, 9)

In [115]:
inv_list = inv_list.where(inv_list > np.quantile(inv_list['invest_num'], 0.25), np.quantile(inv_list['invest_num'], 0.25))

### Document vector

In [116]:
paragraph_id = {k:v for k, v in enumerate(d2v_model.docvecs.doctags.keys())}
doc_vec = {paragraph_id[k]:list(v) for k,v in enumerate(d2v_model.docvecs.vectors_docs)}

In [117]:
inv_tech_df = pd.DataFrame(columns = ['investor', 'tech', 'weights', 'company'])
for values_list in  final_list[['company', 'predicted_small_label1', 'predicted_small_label2', 'invest']].values:
#     print(values_list)
    for com in values_list[-1]:
#         print(com)
        if com.strip() in ['nan', '비공개', '크라우드펀딩', '사모펀드']:
            continue
        else:
            try:
                weights = inv_list.loc[com.strip()].values[0]
                inv_tech_df = inv_tech_df.append({'investor':com.strip(), 'tech':values_list[1], 'weights':weights, 'company':values_list[0]}, \
                                                 ignore_index=True)
            except KeyError:
                print(com)
                continue

유베리타스에프앤아이
 마스트 법률사무소
SL인터베스트
 기술신용보증기금
 인터베스트
 메디치인베스트먼트
 한국보건산업진흥원
 BNH인터베스트
 KB인베스트먼트
킹슬리 벤처스
 엔엑스벤처파트너
디에스씨인베스트먼트
 아이엠엠인베스트먼트
 컴퍼니케이파트너스
 Huayi Investment
 대성창업투자
 ES인베스터
 한국성장금융투자운용
 NP Equity Partners
 프리미어 파트너스
(주)엔비즈닷넷
 중소기업진흥공단
 서울투자파트너스
 AJ캐피탈파트너스
 아이디벤처스
 한빛인베스트먼트
 인터베스트
 아이엠엠인베스트먼트
 라인벤처스
 티에스인베스트먼트
 조명수
 아이디벤처스
 미래에셋캐피탈
NVC파트너스
 컴퍼니케이파트너스
 케이런벤처스
 TBT
 크레스코레이크파트너스
 DS자산운용
 넵튠
피앤아이인베스트먼트
 카카오페이지
 디에스씨인베스트먼트
SAP
 테크스타즈
 500 스타트업스
라인플러스
사이버에이전트벤처즈 코리아
 PIA
 Mission Capital
 GC녹십자Genome
 스파크랩
 데일리파트너스
 원익
옐로금융그룹
 KB인베스트먼트
 한화인베스트먼트
옐로우독
 콜라보레이티브펀드
 알토스벤처스
현대기술투자
 비씨카드(BC카드)
 UTC인베스트먼트
 스마일게이트인베스트먼트
 미래에셋벤처투자
 KT인베스트먼트
 미래에셋벤처투자
카이트창업가재단
킹슬리 벤처스
 GS홈쇼핑
 DS앤파트너스
 뮤렉스파트너스
 아이디지캐피탈파트너스코리아
 케이비인베스트먼트
 아이디벤처스
 현대자동차 제로원엑셀러레이터
케이비인베스트먼트
에이티넘인베스트먼트
 신용보증기금 경기스타트업지점
 헤시드


In [146]:
inv = list(inv_list.index)
inv_idx = ["i%s" % str(i).zfill(3) for i in range(1,len(inv_list.index)+1)]
inv_node = pd.DataFrame({'node':inv,'node_idx':inv_idx})

In [147]:
inv_tech_df.rename(columns={'investor':'node'}, inplace=True)

In [148]:
inv_tech_df.to_excel('./last/investor_graph.xlsx', index=False, encoding='cp949')

### 네트워크

In [149]:
def network_score(G) :
    tt =  { "degree":dict(G.degree()), 
            "degree_centrality":nx.degree_centrality(G), #normalized degree, degree가 클수록 높음
            "closeness_centrality":nx.closeness_centrality(G), 
            "betweenness_centrality":nx.betweenness_centrality(G), 
            "pagerank":nx.pagerank(G)}
    tt_df = pd.DataFrame(tt)
    tt_df['node'] = list(tt_df.index)
    tt_df = pd.merge(tt_df,node_df,how='left')
    return tt_df

In [150]:
small_label = list(set(final_list.predicted_small_label1.tolist() + final_list.predicted_small_label2.tolist()))
small_label = [label for label in small_label if label != 'nan']
comb_dv = list(itertools.combinations(small_label,2))

In [151]:
%%time
tech_similarity_list = list()
tech_mat = pd.DataFrame(index=small_label, columns=small_label)
for comb1, comb2 in comb_dv:
    cos_sim = cosine_similarity([doc_vec[comb1]], [doc_vec[comb2]])[0][0]
    tech_mat.loc[comb1,comb2] = cos_sim
    tech_mat.loc[comb2,comb1] = cos_sim
    tech_similarity_list.append(cos_sim)
tech_mat2 = tech_mat.where(tech_mat > 0.13, 0.0)
tech_mat3 = tech_mat2.where(tech_mat2 == 0, 1)

Wall time: 605 ms


In [152]:
t_edge_df = pd.DataFrame(columns= ['source', 'target', 'weights'])
for idx, comb in enumerate(comb_dv):
    edge =tech_mat3.loc[comb[0], comb[1]]
    if edge == 0.0:
        continue
    else:
        weight = tech_mat2.loc[comb[0], comb[1]]
        t_edge_df = t_edge_df.append({'source':comb[0], 'target':comb[1], 'weights':weight}, ignore_index=True)
t_edge = [tuple(element) for element in t_edge_df.values]

In [153]:
%%time
company = list(final_list['company'])
company_idx = ["c%s" % str(i).zfill(3) for i in range(1,len(company)+1)]
node_df = pd.DataFrame({'node':company,'node_idx':company_idx, 'token':final_list.token.tolist()})
sim_sentence = pd.DataFrame(columns=company, index=company)
comb_c = list(itertools.combinations(node_df.node.tolist(),2))
similarity_list = list()
for idx,comb in enumerate(comb_c):
    if idx % 100 == 0:
        print(comb[0], comb[1])
    sent1 = node_df.loc[node_df.node == comb[0], 'token'].values[0]
    sent2 = node_df.loc[node_df.node == comb[1], 'token'].values[0]
    sent1_vector = d2v_model.infer_vector(sent1)
    sent2_vector = d2v_model.infer_vector(sent2)
    sim_sentence.loc[comb[0], comb[1]] = cosine_similarity([sent1_vector], [sent2_vector])[0][0]
    sim_sentence.loc[comb[1], comb[0]] = cosine_similarity([sent2_vector], [sent1_vector])[0][0]
    similarity_list.append(cosine_similarity([sent1_vector], [sent2_vector])[0][0])
sim_sentence2 = sim_sentence.where(sim_sentence > np.quantile(similarity_list, 0.90), 0.0)
sim_sentence3 = sim_sentence2.where(sim_sentence2 == 0, 1)

케이팝유나이티드 (주)제이엘케이인스펙션
케이팝유나이티드 NICE비즈니스플랫폼
(주)제이엘케이인스펙션 NICE비즈니스플랫폼
(주)뤼이드 소셜네트워크마케팅플랫폼
소셜네트워크마케팅플랫폼 밸런스히어로
착한텔레콤 코노랩스
다이닝코드 트래블플랜
밸런스히어로 매스프레소
비비비 앤톡
샌드박스 네트워크 휴먼스케이프
마이셀럽스 굳센컴퍼니
코노랩스 스페이스워크
리니어블(주) 제네시스랩
스튜디오씨드코리아 뷰티패스
(주)위즈벤처스 슈퍼브에이아이
트래블플랜 텍스트팩토리
제노플랜 휴먼스케이프
어니스트펀드 마켓잇
렌딧 메디픽셀
펀다 와이즈셀렉션
매스프레소 리뷰쉐어
주식회사 날비컴퍼니 마켓잇
클래스101 알리버2017
트위니 NICE비즈니스플랫폼
데이터앤애널리틱스 플레인베이글
밍글 뷰티패스
스켈터랩스 이제이엔
티엔디엔 엘핀
리뷰쉐어 아이팩토리
액션파워 아모랩
베이글랩스 주식회사 셰어킴
휴먼스케이프 제네시스랩
뉴로핏 캐치잇플레이
콴텍 이콜트리
캐치잇플레이 아모랩
한국신용데이터 로지스팟
올핀 와이즈셀렉션
로지스팟 송아리아이티
언더폭스 알리버2017
프리윌린 하우투비즈랩
서큘러스 뷰티패스
레이지앤트 주식회사 스파이스웨어
먼치팩토리 아이팩토리
산타 크레이티버
아모랩 슈퍼브에이아이
닥터다이어리 이콜트리
리턴박스 뷰티패스
크레이티버 주식회사 스파이스웨어
딥네츄럴 하우투비즈랩
뷰티패스 송아리아이티
두브레인 소셜클럽
와이즈셀렉션 주식회사 셰어킴
Wall time: 1min 14s


In [154]:
sim_sentence3

,케이팝유나이티드,(주)제이엘케이인스펙션,콰라,(주)뤼이드,소셜네트워크마케팅플랫폼,착한텔레콤,다이닝코드,밸런스히어로,비비비,샌드박스 네트워크,...,슈퍼브에이아이,패스트비코리아,비링크헬스케어,(주)블루웨일컴퍼니,주식회사 셰어킴,아이팩토리,보이스루,베드라디오,알고리마,NICE비즈니스플랫폼
케이팝유나이티드,0,0,0,0,0,1,1,0,0,1,...,0,0,0,0,1,0,1,0,0,0
(주)제이엘케이인스펙션,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
콰라,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
(주)뤼이드,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
소셜네트워크마케팅플랫폼,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
아이팩토리,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
보이스루,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
베드라디오,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
알고리마,0,0,1,0,0,0,1,0,0,0,...,1,0,1,0,0,0,1,0,0,0


In [155]:
edge_df = pd.DataFrame(columns= ['source', 'target', 'weights'])
for idx,comb in enumerate(comb_c):
    edge =sim_sentence3.loc[comb[0], comb[1]]
    if edge == 0.0:
        continue
    else:
        weight = sim_sentence2.loc[comb[0], comb[1]]
        edge_df = edge_df.append({'source':comb[0], 'target':comb[1], 'weights':weight}, ignore_index=True)
c_edge = [tuple(element) for element in edge_df.values]

In [156]:
small = list(set(list(final_list['predicted_small_label1']) + list(final_list['predicted_small_label2'])))
small_idx = ["s%s" % str(i).zfill(2) for i in range(1,len(small)+1)]
small_node = pd.DataFrame({'node':small,'node_idx':small_idx})
node_df = node_df.append(pd.DataFrame(data=small_node),ignore_index=True)

In [157]:
s_edge = []
for i in range(len(final_list)):
    company = final_list['company'][i]
    small1 = final_list['predicted_small_label1'][i]
    if small1 == 'nan':
        print(company)
    else:
        s_edge.append((small1, company,1))
    small2 = final_list['predicted_small_label2'][i]
    if small2 == 'nan':
        print(company)
    else:
        s_edge.append((small2, company,1))

스튜디오씨드코리아
로지스팟
리턴박스
쉐어트리츠
스터디짐
이콜트리
바이트
아이팩토리


In [198]:
s_edge

[('지능형 O2O 서비스 데이터 분석 시스템', '케이팝유나이티드', 1),
 ('e-커머스 플랫폼', '케이팝유나이티드', 1),
 ('영상 데이터 기반 AI 서비스', '(주)제이엘케이인스펙션', 1),
 ('스마트 헬스 데이터 분석', '(주)제이엘케이인스펙션', 1),
 ('인간-인공지능 협업 시스템', '콰라', 1),
 ('핀테크 빅데이터 분석 및 활용서비스', '콰라', 1),
 ('에듀테크 콘텐츠', '(주)뤼이드', 1),
 ('인공지능 기반 SW', '(주)뤼이드', 1),
 ('SNS 플랫폼', '소셜네트워크마케팅플랫폼', 1),
 ('e-커머스 플랫폼', '소셜네트워크마케팅플랫폼', 1),
 ('e-커머스 플랫폼', '착한텔레콤', 1),
 ('O2O 서비스플랫폼', '착한텔레콤', 1),
 ('빅데이터 기반 SW', '다이닝코드', 1),
 ('외래관광객용 관광 O2O 서비스', '다이닝코드', 1),
 ('국내외 간편송금 및 결제', '밸런스히어로', 1),
 ('모바일 핀테크 보안', '밸런스히어로', 1),
 ('헬스케어 디자인', '비비비', 1),
 ('치료 수술용 기구 및 기기', '비비비', 1),
 ('실감방송 콘텐츠', '샌드박스 네트워크', 1),
 ('동영상 미디어 플랫폼', '샌드박스 네트워크', 1),
 ('Robotic Process Automation', '마이셀럽스', 1),
 ('빅데이터 기반 SW', '마이셀럽스', 1),
 ('빅데이터 기반 SW', '코노랩스', 1),
 ('인간-인공지능 협업 시스템', '코노랩스', 1),
 ('자연어 처리기반 텍스트 마이닝', '리니어블(주)', 1),
 ('스마트 헬스 웨어러블 기기', '리니어블(주)', 1),
 ('e-커머스 플랫폼', '스튜디오씨드코리아', 1),
 ('빅데이터 기반 SW', '(주)위즈벤처스', 1),
 ('SNS 플랫폼', '(주)위즈벤처스', 1),
 ('빅데이터 기반 SW', '스마트토이', 1),
 ('스마트 완구', '스마트토이'

In [199]:
company_tech = final_list[['company','predicted_label1','predicted_label2', 'predicted_small_label1', 'predicted_small_label2']]
company_tech.rename(columns={'company':'node'}, inplace=True)

C:\Users\lmhoon012\Anaconda3\lib\site-packages\pandas\core\frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [200]:
node_df = node_df.append(pd.DataFrame(data=inv_node),ignore_index=True)
node_df.to_excel('./last/new_node_df2.xlsx', index=False, encoding='cp949')

In [201]:
i_edge = [(el[0], el[3],float(el[2])) for el in inv_tech_df.values]

In [202]:
i_edge

[('기술보증기금', '케이팝유나이티드', 0.0547752808988764),
 ('파트너스인베스트먼트', '(주)뤼이드', 0.08286516853932584),
 ('알펜루트자산운용', '(주)뤼이드', 0.025280898876404494),
 ('한국투자파트너스', '(주)뤼이드', 0.2191011235955056),
 ('신용보증기금', '소셜네트워크마케팅플랫폼', 0.07022471910112359),
 ('캡스톤파트너스', '착한텔레콤', 0.17134831460674158),
 ('마젤란기술투자', '다이닝코드', 0.05056179775280899),
 ('포스코기술투자', '다이닝코드', 0.11516853932584269),
 ('소프트뱅크벤처스', '밸런스히어로', 0.14747191011235955),
 ('KDB산업은행', '밸런스히어로', 0.1741573033707865),
 ('캡스톤파트너스', '밸런스히어로', 0.17134831460674158),
 ('네이버', '밸런스히어로', 0.07865168539325842),
 ('메가인베스트먼트', '밸런스히어로', 0.06601123595505617),
 ('신한은행', '밸런스히어로', 0.02949438202247191),
 ('포스코기술투자', '비비비', 0.11516853932584269),
 ('KDB산업은행', '비비비', 0.1741573033707865),
 ('HB인베스트먼트', '비비비', 0.08848314606741572),
 ('신한금융투자', '마이셀럽스', 0.05758426966292135),
 ('TIPS 프로그램', '리니어블(주)', 0.9999999999999999),
 ('본엔젤스벤처파트너스', '리니어블(주)', 0.1699438202247191),
 ('동문파트너즈', '리니어블(주)', 0.06741573033707865),
 ('에버그린투자파트너스', '스튜디오씨드코리아', 0.021067415730337078),
 ('소프트뱅크

In [203]:
total_edge = c_edge+s_edge+i_edge

In [204]:
network_df = pd.DataFrame(total_edge, columns=['Source', 'Target', 'Weight'])

In [205]:
network_df.to_csv('./last/gephi/gephi_edge.csv', index=False, encoding='utf-8')

In [206]:
network_df

,Source,Target,Weight
0,케이팝유나이티드,착한텔레콤,0.328086
1,케이팝유나이티드,다이닝코드,0.240253
2,케이팝유나이티드,샌드박스 네트워크,0.328296
3,케이팝유나이티드,스마트토이,0.268920
4,케이팝유나이티드,어니스트펀드,0.330132
...,...,...,...
877,제주창조경제혁신센터,베드라디오,0.016854
878,신한금융투자,알고리마,0.057584
879,매쉬업엔젤스,알고리마,0.122191
880,TIPS 프로그램,알고리마,1.000000


### Graph(T-C-I)

In [167]:
nondirect_G = nx.Graph()
nondirect_G.add_weighted_edges_from(c_edge+s_edge+i_edge)
nondirect_score = network_score(nondirect_G)
nondirect_score.to_excel('./last/network_score.xlsx', index=False, encoding='cp949')

In [168]:
nondirect_G.nodes

NodeView(('케이팝유나이티드', '착한텔레콤', '다이닝코드', '샌드박스 네트워크', '스마트토이', '어니스트펀드', '(주)삼공일 - 피나포레', '굳센컴퍼니', '마켓잇', '먼치팩토리', 'ERGO', '산타', '닥터다이어리', '소셜클럽', '(주)법정문서', '주식회사 셰어킴', '보이스루', '(주)제이엘케이인스펙션', '비비비', '제노플랜', '한국신용데이터', '머니브레인', '브이픽스메디칼', '루티헬스', '딥네츄럴', '비링크헬스케어', '콰라', '코노랩스', '(주)위즈벤처스', '주식회사 날비컴퍼니', '데이터앤애널리틱스', '밍글', '에임', '제네시스랩', '리턴박스', '메저차이나', '스터디짐', '두브레인', '슈퍼브에이아이', '알고리마', '(주)뤼이드', '클래스101', '콴텍', '오픈핑거스', '소셜네트워크마케팅플랫폼', '스튜디오씨드코리아', '매스프레소', '트위니', '펫프렌즈', '스켈터랩스', '휴먼스케이프', '캐치잇플레이', '언더폭스', '바오바브코리아', '서큘러스', '레이지앤트', '크레이티버', '머지홀딩스', '밸런스히어로', '티엔디엔', '텍스트팩토리', '액션파워', '프리윌린', '송아리아이티', '플레인베이글', '뷰티패스', '페이플', '리니어블(주)', '올핀', '마이셀럽스', '이제이엔', '쉐어트리츠', '트래블플랜', '렌딧', '알고리즘랩스', '이콜트리', '마스터지지', '(주)블루웨일컴퍼니', '하우투비즈랩', '체크멀', '3billion', '펀다', 'NICE비즈니스플랫폼', '바이트', '리뷰쉐어', '엘핀', '아이팩토리', '앤톡', '베이글랩스', '베라노스', '패스트비코리아', '체커', '메디픽셀', '베드라디오', '알리버2017', '뉴로핏', '주식회사 스파이스웨어', '스페이스워크', '와이즈셀렉션', '로지스팟', '지능형 O2O 서비스 데이터 분석 시스템', 'e-커머스 플랫폼', '영상 데이터 기반 AI 서비스', '스

In [170]:
nondirect_score

,degree,degree_centrality,closeness_centrality,betweenness_centrality,pagerank,node,node_idx,token
0,19,0.077869,0.388535,0.025331,0.007591,케이팝유나이티드,c001,"[웹서비스, 마케팅, pop, united, 만들어, 글로벌, pop, 커뮤니티, ..."
1,21,0.086066,0.396748,0.022123,0.008135,착한텔레콤,c006,"[서비스, 웹서비스, 마케팅, 소프트웨어, 플랫폼, 커머스, 오픈마켓, 휴대폰, 휴..."
2,21,0.086066,0.396748,0.025702,0.007456,다이닝코드,c007,"[다이닝코드, 빅데이터, 맛집, 모바일앱, 서비스, 웹서비스, 마케팅, 소프트웨어,..."
3,11,0.045082,0.368580,0.017232,0.005911,샌드박스 네트워크,c010,"[서비스, 웹서비스, 마케팅, 소프트웨어, 데이터, 엔터테인먼트, mcn, 유튜브,..."
4,5,0.020492,0.319372,0.008613,0.003812,스마트토이,c016,"[소개, 전문인력, 양성, 사업, 구성, 사업, 진행, 실험, 새로, 블록, 설계,..."
...,...,...,...,...,...,...,...,...
245,1,0.004098,0.274157,0.000000,0.000670,유안타인베스트먼트,i121,NaN
246,1,0.004098,0.271413,0.000000,0.000621,고려대학교기술지주,i086,NaN
247,1,0.004098,0.249489,0.000000,0.000645,중소벤처기업부,i051,NaN
248,1,0.004098,0.289100,0.000000,0.000703,더인벤션랩,i035,NaN


#### NTBF

In [171]:
company_network_result = nondirect_score[nondirect_score['node_idx'].isin(company_idx)].sort_values(["pagerank"], ascending=[False])
company_network_result['total_weight'] = company_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
company_network_result = company_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
company_network_result = pd.merge(company_network_result, company_tech)
company_network_result.to_excel('./last/company_network_result.xlsx', index=False, encoding='cp949')

In [172]:
com_score1 = company_network_result.groupby('predicted_small_label1').mean()[['total_weight']].sort_values('total_weight', ascending=False)

In [173]:
com_score2 = company_network_result.groupby('predicted_small_label2').mean()[['total_weight']].sort_values('total_weight', ascending=False)

In [174]:
com_score2.rename(columns ={'total_weight':'total_weight2'}, inplace=True)

In [175]:
total_index = list(set(com_score1.index.tolist() + com_score2.index.tolist()))

In [176]:
company_total_df = pd.DataFrame(index=total_index)

In [177]:
com_score1.index.name = None
com_score2.index.name = None

In [178]:
company_total_df = company_total_df.join(com_score1)
company_total_df = company_total_df.join(com_score2)

In [179]:
company_total_df.fillna(0, inplace=True)

In [180]:
company_score = pd.DataFrame(company_total_df.mean(axis=1), columns=['total_weight']).sort_values('total_weight', ascending=False)

In [181]:
company_score.to_excel('./last/company_based_score.xlsx')

#### Emerging Technology

In [182]:
tech_network_result = nondirect_score[nondirect_score['node_idx'].isin(small_idx)].sort_values(["pagerank"], ascending=[False])
tech_network_result['total_weight'] = tech_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
tech_network_result = tech_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
tech_network_result.to_excel('./last/tech_network_result.xlsx', index=False, encoding='cp949')

In [183]:
tech_network_result[['node', 'total_weight']].sort_values('total_weight', ascending=False).to_excel('./last/emerging_tech.xlsx', index=False)

In [184]:
tech_score = tech_network_result[['node', 'total_weight']].sort_values('total_weight', ascending=False).set_index('node')

In [185]:
tech_score.index.name=None

#### investor

In [186]:
inv_network_result = nondirect_score[nondirect_score['node_idx'].isin(inv_idx)].sort_values(["pagerank"], ascending=[False])
inv_network_result['total_weight'] = inv_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
inv_network_result = inv_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
inv_network_result = pd.merge(inv_network_result, inv_tech_df)
inv_network_result.to_excel('./last/inv_network_result.xlsx', index=False, encoding='cp949')

In [187]:
inv_score = inv_network_result.groupby('tech').mean()[['total_weight']]

In [188]:
inv_score.sort_values('total_weight', ascending=False).to_excel('./last/inv_tech.xlsx')

In [189]:
inv_score.index.name=None

#### Total weight

In [190]:
total_weight_index = list(set(company_score.index.tolist() + inv_score.index.tolist() + tech_network_result.node.tolist()))
total_score = pd.DataFrame(index=total_weight_index)

In [191]:
company_score.rename(columns={'total_weight':'NS'}, inplace=True)
tech_score.rename(columns={'total_weight':'TS'}, inplace=True)
inv_score.rename(columns={'total_weight':'IS'}, inplace=True)

In [192]:
company_score.to_excel('./last/NS.xlsx')
tech_score.to_excel('./last/TS.xlsx')
inv_score.to_excel('./last/IS.xlsx')

In [193]:
total_score = total_score.join(company_score)
total_score = total_score.join(tech_score)
total_score = total_score.join(inv_score)

In [194]:
total_score_df = pd.DataFrame(total_score.fillna(0).sum(axis=1), columns=['Total score']).sort_values('Total score', ascending=False)

In [195]:
total_score_df.to_excel('./last/TOS.xlsx')